# Initialisation du système

**a) Réseau discussion:**  
Nous initialisons **une matrice** de $ N\_1dim*N\_1dim=N $ individus. De plus, nous modélisons un réseau discussion par **un dictionaire**, qui associe à chaque individu un tuple, de la forme $ dict[str:tuple[int,bool,float,float]] $. Celui-ci représentera intégralement un agent et ses caractéristiques.

On rappelle la description d'un agent : 

|Caractéristique||Nom    |Type   |Intervalle     |Valeur initiale  | Fixe ?    |
|:-------------:||:-----:|:-----:|:-------------:|:---------------:|:---------:|
|**Nom** : nom de l'agent (clé du dict)                                  ||**nom**    |str    |[str(0),str(N)]|str(k)|Oui|
|**Opinion** : y croit (1) ou non (0)                                    ||**op**     |int    |[0,1]          |0     |Non|
|**Déjà vu** : nb d'expositions à la news                                ||**deja_vu**|int   |[0,steps]   |0 |Non|
|**Force de persuasion** : capacité à convaincre qqun (accrue qd tend vers 1) ||**fp**|float  |[0,1]          |Random|Oui|
|**Conviction** : caractérise la conviction à la news (tend vers 1) ou non (tend vers 0)||**p**|float  |[0,1] |0.5   |Non| 

In [94]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import random

#description d'un agent : nom:(opinion, deja_vu , force_de_persuasion , conviction)
#                         nom:(  op   , deja_vu ,       fp            ,     p     )

#VARIABLES GLOBALES / PARAMETRES DU MODELE :
N_1dim = 5                #nb d'agents sur une dimension
N = N_1dim**2             #nb d'agents total
nb_om = 10                 #nb d'organismes médiatiques (malveillants ou non)
nb_xom = 4                #nb d'organismes médiatiques malveillants
mal = nb_xom/nb_om        #degré de malveillance (= proportion de médias malveillants)
co = 7                    #degré de connectivité de l'environnement (= nb de réseaux moyen par agent = somme(nb_res) / nb_agents)
steps = 100               #nb de pas de temps
news = np.random.random() #crédibilité de la news
seuil = 0.7               #seuil

print('Crédibilité de la news :',news,'\n')

#Initialisation d'un réseau d'humains :
Reseau_discu={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Noms_reseau_discu=[str(k) for k in range(1,N+1)]

#Initialisation d'un réseau avec des agents malveillants (temporaire)
nb_malv=6
def init_sys_avec_malv(n_1dim,nb_malv):
    
    Pop={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
    L=np.random.choice(range(1,(n_1dim**2)+1), size=nb_malv, replace=False)
    for e in L:
        Pop[str(e)]=(1,0,0.9,0.8)
    
    return Pop

Pop_malv=init_sys_avec_malv(N_1dim,nb_malv)
print('Réseau avec malveillants : \n',Pop_malv,'\n')

#Initialisation d'un système avec organismes médiatiques (malveillants et neutres):
List_om=np.random.choice(range(1,N+1), size=nb_om, replace=False)
print('Liste des médias :',List_om,'\n')

List_xom=np.random.choice(List_om, size=nb_xom, replace=False)
print('Liste des médias malveillants :',List_xom,'\n')

def init_ac_om_et_xom(reseau,list_om,list_xom):
    for name in reseau:
        if 'O' not in name and 'X' not in name:
            if int(name) in list_om and int(name) not in list_xom:
                reseau['OM'+str(name)]=(0,0,np.random.uniform(low=0.7,high=1.0),0.5)
                del reseau[name]
            if int(name) in list_xom:
                reseau['XOM'+str(name)]=(0,0,np.random.uniform(low=0.7,high=1.0),0.75)
                del reseau[name]
                
    return reseau

Pop_ac_om=init_ac_om_et_xom(Reseau_discu,List_om,List_xom)
print('Population avec médias : \n',Pop_ac_om,'\n')

#Réseau à valeurs quelconques (au milieu de la simulation par exemple) pour faire des tests :
Test_reseau={str(k):(np.random.choice([0,1]),np.random.choice(range(steps)),np.random.random_sample(),np.random.random_sample()) for k in range(1,N+1)}

#Liste/matrice des noms des agents :
    #Matrice sans médias
Noms_agents_sans=[name for name in Pop_malv]
np.random.shuffle(Noms_agents_sans)
Mat_pop_sans_om=np.reshape(Noms_agents_sans,(N_1dim,N_1dim)).copy()
print('Matrice agents (sans médias) :\n',Mat_pop_sans_om,'\n')
    #Matrice avec médias
Noms_agents_avec=[name for name in Pop_ac_om]
np.random.shuffle(Noms_agents_avec)
Mat_pop_ac_om=np.reshape(Noms_agents_avec,(N_1dim,N_1dim)).copy()
print('Matrice agents (avec médias) :\n',Mat_pop_ac_om,'\n')

#Matrice des opinions de départ :
def matrice_opinions(reseau,names,n_1dim):
    Op=np.zeros((n_1dim,n_1dim))
    i=0
    j=0
    for nom in names:
        op,deja,fp,p=reseau[nom]
        Op[i,j]=op
        if j<(n_1dim-1):
            j=j+1
        elif j==(n_1dim-1):
            j=0
            i=i+1
    return Op

print('Matrice op (avec médias) :\n',matrice_opinions(Pop_ac_om,Noms_agents_avec,N_1dim),'\n')

#Matrice des convictions de départ
def matrice_convic(reseau,names,n_1dim):
    P=np.zeros((n_1dim,n_1dim))
    i=0
    j=0
    for nom in names:
        op,deja,fp,p=reseau[nom]
        P[i,j]=p
        if j<(n_1dim-1):
            j=j+1
        elif j==(n_1dim-1):
            j=0
            i=i+1
    return P

print('Matrice p (avec médias) :\n',matrice_convic(Pop_ac_om,Noms_agents_avec,N_1dim),'\n')

Crédibilité de la news : 0.29036307407911044 

Réseau avec malveillants : 
 {'1': (0, 0, 0.32385187183498343, 0.5), '2': (1, 0, 0.9, 0.8), '3': (1, 0, 0.9, 0.8), '4': (0, 0, 0.63470309018813142, 0.5), '5': (1, 0, 0.9, 0.8), '6': (0, 0, 0.82569097697630867, 0.5), '7': (0, 0, 0.71110324268437086, 0.5), '8': (0, 0, 0.029162304913036841, 0.5), '9': (0, 0, 0.72971229343774036, 0.5), '10': (0, 0, 0.88172543268995551, 0.5), '11': (0, 0, 0.23926011383918311, 0.5), '12': (0, 0, 0.071082466211361717, 0.5), '13': (1, 0, 0.9, 0.8), '14': (0, 0, 0.28320451098384336, 0.5), '15': (0, 0, 0.15382359024600922, 0.5), '16': (0, 0, 0.51833450333212894, 0.5), '17': (0, 0, 0.4446847768629667, 0.5), '18': (0, 0, 0.44621150578120561, 0.5), '19': (0, 0, 0.74628775053372043, 0.5), '20': (0, 0, 0.23833877440582552, 0.5), '21': (0, 0, 0.10224294733681172, 0.5), '22': (1, 0, 0.9, 0.8), '23': (0, 0, 0.005587080611165085, 0.5), '24': (1, 0, 0.9, 0.8), '25': (0, 0, 0.93806515310924798, 0.5)} 

Liste des médias : [17 2

**b) Réseaux sociaux :**  
Nous modéliserons les **10 réseaux sociaux les plus utilisés en 2017** (source : http://www.influenth.com/top-10-reseaux-sociaux-2017/) par un dictionnaire du type $ dict[str:list[str]] $ qui à un nom de réseau social associe la liste de personnes inscrites.
Nous modéliserons un réseau virtuel par **un dictionnaire** qui à chaque agent (humain ou organisme médiatique) associe une liste d'amis ou d'abonnés. Il représentera la possibilité d'interaction.

In [95]:
#RESEAUX SOCIAUX
L_reseaux=['facebook','qzone','instagram','tumblr','twitter','baidu','weibo','snapchat','pinterest','linkedin']

#Nb de réseaux par agent:
def nb_res(co,n):
    k=0
    L=[]

    while k<n-1:
        if k%2==0:
            if co<5:
                nb=np.random.choice(range(2*co+1))
            else:
                nb=np.random.choice(range(11))
            L.append(nb)
        else:
            if 2*co-nb<=10:
                L.append(2*co-nb)
            else:
                L[k-1]=L[k-1]+(2*co-nb-10)
                L.append(10)
        k=k+1
    if n%2==1:
        L.append(co)
    else:
        if 2*co-nb<=10:
            L.append(2*co-nb)
        else:
            L[k-1]=L[k-1]+(2*co-nb-10)
            L.append(10)
        
    return L

Nb_res={name : nb for (name,nb) in zip(Noms_agents_avec,nb_res(co,N))}
print('Nb de réseaux par agent :\n',Nb_res,'\n')

#Agents et les réseaux auxuels ils appartiennent:
Agents_ac_reseaux={name : list(np.random.choice(L_reseaux, size=Nb_res[name], replace=False)) for name in Noms_agents_avec}
print('Agents et les réseaux auxquels ils appartiennent :\n',Agents_ac_reseaux,'\n')

#Réseaux et leurs utilisateurs:
def res_et_ag(ag_res,reseaux):
    D=dict()
    for res in reseaux:
        D[res]=[]
        for ag in ag_res:
            if res in ag_res[ag]:
                D[res].append(ag)
    return D
    
Reseaux_ac_agents=res_et_ag(Agents_ac_reseaux,L_reseaux)
print('Reseaux et leurs utilisateurs :\n',Reseaux_ac_agents,'\n')

#Réseau virtuel (dico liste amis)
def reseau_virtu(agents_et_res,res_et_agents,nb_res):
    D=dict()
    for a in agents_et_res:
        if nb_res[a]==0:
            D[a]=[]
        else:
            for r in agents_et_res[a]:
                Amis=set(np.random.choice(res_et_agents[r],size=np.random.choice(range(len(res_et_agents[r])))))
                if a in Amis:
                    Amis.remove(a)
                D[a]=list(Amis)    
    for a in D:
        for ami in D[a]:
            if a not in D[ami] and nb_res[ami]!=0:
                D[ami].append(a)
    return D

Reseau_virtuel=reseau_virtu(Agents_ac_reseaux,Reseaux_ac_agents,Nb_res)
print('Réseau virtuel :\n',Reseau_virtuel)

Nb de réseaux par agent :
 {'14': 4, '18': 10, 'XOM5': 6, 'OM24': 8, '16': 4, 'OM17': 10, 'XOM23': 4, 'XOM25': 10, '4': 8, 'XOM3': 6, 'OM1': 4, '7': 10, 'OM8': 4, '6': 10, '2': 5, '15': 9, '10': 4, 'OM11': 10, '21': 9, '20': 5, '12': 10, 'OM13': 4, '19': 9, '22': 5, '9': 7} 

Agents et les réseaux auxquels ils appartiennent :
 {'14': ['pinterest', 'facebook', 'tumblr', 'qzone'], '18': ['weibo', 'instagram', 'qzone', 'tumblr', 'linkedin', 'pinterest', 'baidu', 'twitter', 'snapchat', 'facebook'], 'XOM5': ['qzone', 'linkedin', 'instagram', 'tumblr', 'baidu', 'weibo'], 'OM24': ['qzone', 'weibo', 'snapchat', 'tumblr', 'twitter', 'linkedin', 'facebook', 'instagram'], '16': ['tumblr', 'facebook', 'qzone', 'twitter'], 'OM17': ['tumblr', 'baidu', 'qzone', 'snapchat', 'weibo', 'twitter', 'facebook', 'instagram', 'linkedin', 'pinterest'], 'XOM23': ['snapchat', 'pinterest', 'facebook', 'baidu'], 'XOM25': ['tumblr', 'weibo', 'qzone', 'facebook', 'linkedin', 'snapchat', 'twitter', 'pinterest', 'baid

# DISCUSSION PHYSIQUE

Dans cette première partie, nous nous concentrerons sur la propagation d'une fake news par le seul biais d'une interaction entre agents physiquement proches. Cette discussion pourra avoir lieu **entre 2 à 9 personnes**.

**Processus de discussion détaillé :** 
- **1ère étape : Mise à jour des convictions**  
A partir d'une matrice représentant l'ensemble de la population, nous sélectionnerons un agent et son voisinage pour créer une liste de 2 à 9 personnes : la **liste discussion**.
    - si il n'y a que _2 personnes_ dans la liste discussion, seule la **force de persuasion** influera et ce sur un seul agent, celui dont la force de persuasion est la plus faible. La mise à jour des convictions est alors régie par la loi suivante : lorsque $  fp_i $ < $  fp_j $ on applique la formule: $ p_i \leftarrow p_i + fp_i  \cdot ( p_j - p_i ) $
    - si la discussion comprend _de 3 à 9 personnes_, ce sera le **voisinage** qui influera sur la conviction de chacun selon la loi suivante. Chaque individu modifie sa conviction en fonction de la moyenne des convictions des autres pondérée de leur force de persuasion ($ moy = \frac{\sum p_i*fp_i}{\sum fp_i} $) et du seuil de persuasion $\sigma$ : si $ |p_i-moy| < \sigma $ alors $ p_i \leftarrow moy $ sinon, $p_i$ ne change pas. Nous estimons que seul le voisinage influera dans ce cadre. Comme le corroborent certaines expériences telles que celle de Asch _(explication : https://sciencetonnante.wordpress.com/2013/04/22/lexperience-de-asch-sur-le-conformisme/)_ l'humain tend à se conformer au comportement d'autrui lorsqu'il est entouré. C'est pourquoi nous ne prenons en compte que les convictions des autres et pas la crédibilité de la news (qui entrera en jeu plus tard).  


- **2ème étape : Mise à jour des opinions**  
Pour chaque agent, on modifie son opinion en fonction de sa conviction : si $ p_i  > 0.5 $ , $op_i$ passe de $0$ à $1$ et inversement.

# I-) Facteurs modifiant les paramètres

## 1- Influence du voisinage sur p

### a) Création d'une liste discussion

Chaque individu possède 8 voisins, où qu'il soit. En effet :
- s'il se situe _au milieu de la matrice_ (pas sur les bords ni les coins) ses voisins seront ses voisins directs (en haut, en bas, en haut à gauche, etc...).
- s'il est _sur un bord_ (supérieur par exemple), ses 3 voisins du haut sont remplacés par les 3 agents situés en bas de la matrice correspondant aux mêmes lignes. On sélectionnera ses voisins de manière analogue s'il est sur une autre bordure.
- s'il est _sur un coin_ (supérieur droit par exemple), il possède 3 voisins directs (en bas, à gauche, en bas à gauche) auxquels on ajoutera les agents situés : sur le coin supérieur droit ainsi que son voisin du bas, sur le coin inférieur droit ainsi que son voisin de gauche, sur le coin inférieur gauche.

In [46]:
# Sélection d'un agent et de son voisinage pour établir un discussion
def selection_voisinage(Matrice,n_1dim,a_central):
    '''Sélection aléatoire des voisinages d'un individu
    '''
    L=[]                               #liste voisinage (pers susceptibles de discuter)
    Ldiscu=[]                          #liste discussion (pers qui vont discuter)
    size=np.random.choice(range(2,10)) #nb pers discussion
    
    ic=np.argwhere(Matrice==a_central)[0][0] #ic,jc : indices de l'agent central
    jc=np.argwhere(Matrice==a_central)[0][1]
    
    if ic>0 and ic<(n_1dim-1) and jc>0 and jc<(n_1dim-1):
        L=np.reshape(Matrice[ic-1 : ic+2 , jc-1 : jc+2] , 9)        
    elif ic==0 and jc>0 and jc<(n_1dim-1):          #agent central sur arête sup
        L=[(Matrice[ic,jc-1 : jc+2])
           ,(Matrice[ic+1,jc-1 : jc+2])
           ,(Matrice[n_1dim-1, jc-1 : jc+2])]
        L=np.reshape(L,9)
    elif ic==(n_1dim-1) and jc>0 and jc<(n_1dim-1): #agent central sur arête inf
        L=[(Matrice[ic,jc-1 : jc+2])
           ,(Matrice[ic-1,jc-1 : jc+2])
           ,(Matrice[0, jc-1 : jc+2])]
        L=np.reshape(L,9)        
    elif jc==0 and ic>0 and ic<(n_1dim-1):          #agent central sur arête gauche
        L=[(Matrice[ic-1 : ic+2,jc])
           ,(Matrice[ic-1 : ic+2,jc+1])
           ,(Matrice[ic-1 : ic+2,n_1dim-1])]
        L=np.reshape(L,9)        
    elif jc==(n_1dim-1) and ic>0 and ic<(n_1dim-1): #agent central sur arête droite
        L=[(Matrice[ic-1 : ic+2,jc])
           ,(Matrice[ic-1 : ic+2,jc-1])
           ,(Matrice[ic-1 : ic+2,0])]
        L=np.reshape(L,9)        
    elif ic==0 and jc==0:                           #agent central sur coin sup/gauche
        L=[Matrice[0,n_1dim-1],Matrice[0,0],Matrice[0,1]
           ,Matrice[1,n_1dim-1],Matrice[1,0],Matrice[1,1]
           ,Matrice[n_1dim-1,0],Matrice[n_1dim-1,1],Matrice[n_1dim-1,n_1dim-1]]
    elif ic==0 and jc==n_1dim-1:                    #agent central sur coin sup/droit
        L=[Matrice[ic,jc-1],Matrice[ic,jc],Matrice[ic,0]
          ,Matrice[ic+1,jc-1],Matrice[ic+1,jc],Matrice[ic+1,0]
          ,Matrice[n_1dim-1,jc-1],Matrice[n_1dim-1,jc],Matrice[n_1dim-1,0]]
    elif ic==n_1dim-1 and jc==0:                    #agent central sur coin inf/gauche
        L=[Matrice[ic-1,n_1dim-1],Matrice[ic-1,jc],Matrice[ic-1,jc+1]
          ,Matrice[ic,n_1dim-1],Matrice[ic,jc],Matrice[ic,jc+1]
          ,Matrice[0,n_1dim-1],Matrice[0,0],Matrice[0,1]]
    elif ic==n_1dim-1 and jc==n_1dim-1:             #agent central sur coin inf/droit
        L=[Matrice[ic-1,jc-1],Matrice[ic-1,jc],Matrice[ic-1,0]
          ,Matrice[ic,jc-1],Matrice[ic,jc],Matrice[ic,0]
          ,Matrice[0,jc-1],Matrice[0,jc],Matrice[0,0]]
    
    Ldiscu=np.random.choice(L,size,replace=False)
             
    return Ldiscu

Population=Mat_pop_sans_om
List_discu=selection_voisinage(Population,N_1dim,np.random.choice(Noms_reseau_discu))
print('Population :\n',Population)
print('Liste des personnes qui vont discuter :',List_discu)

Population :
 [['21' '11' '20' '18' '17']
 ['16' '2' '25' '8' '13']
 ['19' '1' '24' '4' '6']
 ['23' '22' '12' '10' '14']
 ['7' '5' '3' '9' '15']]
Liste des personnes qui vont discuter : ['10' '20' '18' '9']


### b) Modification des convictions (paramètre p) des agents de la liste discussion

In [47]:
#Modification du paramètre p de x en fonction de la force de persuasion de y et vice versa :
def force_persu(x, y, reseau):
        
    opx,deja_vux,fpx,px=reseau[str(x)]
    opy,deja_vuy,fpy,py=reseau[str(y)]
    if fpx>fpy:
        py=py+fpx*(px-py)
    elif fpy>fpx:
        px=px+fpy*(py-px)
    
    reseau[str(x)]=(opx,deja_vux+1,fpx,px)
    reseau[str(y)]=(opy,deja_vuy+1,fpy,py)
    
    return reseau


#Influence du voisinage :
    # si 2 personnes, force persu. Sinon, moyenne des p pondérée des fp; si abs(pi-moy)>seuil, pi=pi, sinon pi=moy
def influ_voisinage(list_discu,reseau,seuil):
    
    P=[]   #liste des convictions des membres de la discussion
    F=[]   #liste des forces de persuasion des membres de la discussion
    N=[]   #liste des noms des membres de la discussion
    i=0
    
    for k in list_discu:
        op_k,deja_k,fp_k,p_k=reseau[k]
        N.append(k)
        P.append(p_k)
        F.append(fp_k)
            
    if len(N)==2:
        reseau=force_persu(N[0],N[1],reseau)
    else:
        for a in N:
            op_a,deja_a,fp_a,p_a=reseau[a]
            other_p=P[0:i]+P[i+1:len(P)]
            other_fp=F[0:i]+F[i+1:len(F)]            
            moy=np.average(other_p,weights=other_fp)
            if abs(p_a-moy)<seuil:
                p_a=moy
            if moy!=0.5:
                deja_a=deja_a+1
            reseau[a]=(op_a,deja_a,fp_a,p_a)
            i=i+1
            
    return reseau

print('Réseau avant :',Test_reseau,'\n Discussion :',List_discu)
print('Réseau après :',influ_voisinage(List_discu,Test_reseau,seuil))

Réseau avant : {'1': (0, 91, 0.9427781689706226, 0.1938581001925015), '2': (1, 69, 0.4684467367915893, 0.4770749382854612), '3': (0, 97, 0.9539528521263793, 0.6677623784843598), '4': (1, 83, 0.97917127274754, 0.0018653396184946658), '5': (0, 59, 0.35024128184758396, 0.25906442716520994), '6': (0, 91, 0.401124381202551, 0.9003486473401646), '7': (0, 44, 0.710534237396999, 0.4505880474572559), '8': (0, 84, 0.3426937890042079, 0.36095592685728095), '9': (1, 79, 0.846105599398168, 0.7339100001059322), '10': (1, 73, 0.24621337563091206, 0.530070151395489), '11': (0, 36, 0.20146827598457273, 0.6825385218373279), '12': (1, 20, 0.9210907404067731, 0.3298650699255222), '13': (1, 48, 0.6996532839500865, 0.6793154103047565), '14': (1, 11, 0.026152024238912386, 0.6537486038333145), '15': (0, 25, 0.7754045900865861, 0.6554819644586027), '16': (0, 85, 0.6714491023443734, 0.05434589175567972), '17': (1, 27, 0.8393476271513293, 0.40673427148740293), '18': (0, 50, 0.4409552445834233, 0.8194481710714122

# II-) Simulation

## 1) Itération sur plusieurs pas de temps

Un pas de temps correspond ici à une discussion.

### a) Simulation de l'évolution des opinions

In [48]:
def simulation_op(reseau,n_1dim,mat_pop,names,steps,seuil):
    """retourne la matrice des OPINIONS de la population après un certain nb d'étapes"""
    
    results=[matrice_opinions(reseau,names,n_1dim)]
    
    for simu in range(steps):
        i=np.random.choice(names)
        List_discu=selection_voisinage(mat_pop,n_1dim,i)
        Res_p_changé=influ_voisinage(List_discu,reseau,seuil)
        for c in Res_p_changé:
            op,deja,fp,p=Res_p_changé[c]
            if p>0.5 and op==0:
                op=1
                Res_p_changé[c]=(op,deja,fp,p)
            elif p<0.5 and op==1:
                op=0
                Res_p_changé[c]=(op,deja,fp,p)
        Op=[]
        for e in Res_p_changé:
            op,deja,fp,p=Res_p_changé[e]
            Op.append(op)
        Op_final=np.reshape(Op, (n_1dim,n_1dim))
        results.append(Op_final.copy())
        
    return results

simulation_op(init_sys_avec_malv(N_1dim,nb_malv),N_1dim,Population,Noms_reseau_discu,steps,seuil)

[array([[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  0.,  1.]]), array([[0, 0, 1, 1, 0],
        [0, 0, 0, 1, 1],
        [0, 1, 0, 1, 1],
        [0, 0, 0, 0, 1],
        [0, 1, 0, 1, 1]]), array([[0, 0, 1, 1, 1],
        [0, 1, 0, 1, 1],
        [1, 1, 0, 1, 1],
        [0, 0, 0, 0, 1],
        [1, 1, 1, 1, 1]]), array([[0, 0, 1, 1, 1],
        [0, 1, 0, 1, 1],
        [1, 1, 0, 1, 1],
        [0, 0, 0, 0, 1],
        [1, 1, 1, 1, 1]]), array([[0, 0, 1, 1, 1],
        [0, 1, 0, 1, 1],
        [1, 1, 0, 1, 1],
        [0, 0, 1, 0, 1],
        [1, 1, 1, 1, 1]]), array([[0, 0, 1, 1, 1],
        [0, 1, 0, 1, 1],
        [1, 1, 0, 1, 1],
        [0, 0, 1, 0, 1],
        [1, 1, 1, 1, 1]]), array([[1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1],
        [1, 1, 0, 1, 1],
        [0, 0, 1, 0, 1],
        [1, 1, 1, 1, 1]]), array([[1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1],
        [1, 1, 0, 

### b) Simulation de l'évolution des convictions

In [57]:
def simulation_p(reseau,n_1dim,mat_pop,names,steps,seuil):
    """retourne la matrice des CONVICTIONS de la population après un certain nb d'étapes"""
    
    results=[matrice_convic(reseau,names,n_1dim)]
    
    for simu in range(steps):
        i=np.random.choice(names)
        List_discu=selection_voisinage(mat_pop,n_1dim,i)
        Res_p_changé=influ_voisinage(List_discu,reseau,seuil)
        for c in Res_p_changé:
            op,deja,fp,p=Res_p_changé[c]
            if p>0.5 and op==0:
                op=1
                Res_p_changé[c]=(op,deja,fp,p)
            elif p<0.5 and op==1:
                op=0
                Res_p_changé[c]=(op,deja,fp,p)
        Op=[]
        P=[]
        for e in Res_p_changé:
            op,deja,fp,p=Res_p_changé[e]
            Op.append(op)
            P.append(p)
        Op_final=np.reshape(Op, (n_1dim,n_1dim))
        P_final=np.reshape(P, (n_1dim,n_1dim))
        results.append(P_final.copy())
        
    return results

simulation_p(init_sys_avec_malv(N_1dim,nb_malv),N_1dim,Population,Noms_reseau_discu,steps,seuil)

[array([[ 0.5,  0.8,  0.5,  0.5,  0.5],
        [ 0.8,  0.5,  0.8,  0.5,  0.5],
        [ 0.5,  0.5,  0.5,  0.8,  0.5],
        [ 0.5,  0.5,  0.5,  0.5,  0.5],
        [ 0.8,  0.8,  0.5,  0.5,  0.5]]),
 array([[ 0.5       ,  0.8       ,  0.55918926,  0.5       ,  0.55377919],
        [ 0.8       ,  0.55983008,  0.8       ,  0.5       ,  0.5       ],
        [ 0.5       ,  0.55455576,  0.5       ,  0.8       ,  0.5       ],
        [ 0.5       ,  0.5       ,  0.5       ,  0.55254674,  0.5       ],
        [ 0.8       ,  0.5       ,  0.56065108,  0.55775546,  0.5       ]]),
 array([[ 0.5       ,  0.8       ,  0.55918926,  0.6233849 ,  0.55377919],
        [ 0.57442765,  0.61507898,  0.8       ,  0.60984688,  0.62166046],
        [ 0.5       ,  0.55455576,  0.5       ,  0.57442765,  0.61851517],
        [ 0.5       ,  0.5       ,  0.5       ,  0.60958258,  0.5       ],
        [ 0.8       ,  0.5       ,  0.61557311,  0.55775546,  0.5       ]]),
 array([[ 0.5       ,  0.8       ,  0.593959

## 2) Affichage

### a) Représentation des opinions au début et à la fin de la simulation

In [7]:
Simulation_op=simulation_op(init_sys_avec_malv(N_1dim,nb_malv),N_1dim,Population,Noms_reseau_discu,steps,seuil)

#Affichage des opinions de départ
Opinions_start=Simulation_op[0]
fig=plt.figure()
im=plt.imshow(Opinions_start,cmap='binary',vmin=0,vmax=1)

#Affichage des opinions d'arrivée
Opinions_end=Simulation_op[-1]
fig=plt.figure()
im=plt.imshow(Opinions_end,cmap='binary',vmin=0,vmax=1)

#blanc = 0 ; noir = 1

### b) Représentation des convictions au début et à la fin de la simulation

In [8]:
Simulation=simulation_p(init_sys_avec_malv(N_1dim,nb_malv),N_1dim,Population,Noms_reseau_discu,steps,seuil)

#Affichage des convictions de départ
Convictions_start=Simulation[0]
fig=plt.figure()
im=plt.imshow(Convictions_start,cmap='bwr',vmin=0,vmax=1)
plt.colorbar()

#Affichage des convictions de fin de simulation

Convictions_end=Simulation[-1]
fig=plt.figure()
im=plt.imshow(Convictions_end,cmap='bwr',vmin=0,vmax=1)
plt.colorbar()

# CONSULTATION A DISTANCE

Dans cette deuxième partie, nous nous intéresserons à la transmission d'une fake news à distance, c'est-à-dire par le biais d'un organisme médiatique ou d'un réseau social. Des personnes physiquement éloignées pourront alors interagir.  

**Processus détaillé :**  
Comme évoqué dans la partie "Initialisation du système", le réseau virtuel nous fournit, pour chaque agent, la liste des autres agents qui voient la news si celui-ci la partage (on peut dire que c'est sa liste d'amis ou d'abonnés). Pour faire un parallèle avec la partie "Discussion physique", on peut considérer que c'est l'équivalent de la liste discussion.  
- **1ère étape : lancement de la fake news**  
Un média malveillant est choisi au hasard. Son opinion est alors mise à 1 et il partage la news à sa liste d'abonnés grâce à l'algorithme décrit ci-dessous.
- **2ème étape : partage de la fake news**  
Pour ce qui est du partage de l'information, le processus est semblable à celui de la discussion physique à quelques différences près.  
On met toujours à jour successivement les convictions puis les opinions selon plusieurs facteurs :
    - la **force de persuasion** jouera le même rôle que précédemment. Nous estimons que même à travers un réseau social ou un article, la force de persuasion a son rôle à jouer.
    - contrairement à la discussion physique où la conviction dépend exclusivement du voisinage, lorsqu'un agent se retrouve seul derrière son écran ou son journal, le rôle de la **crédibilité de la news** que nous nommerons "$news$" est prépondérant (on remarquera au passage que cette valeur (comprise entre 0 et 1) est la seule caractéristique propre à la news). La conviction $p$ de l'agent de force de persuasion $fp$ sera modifiée selon la loi suivante : $ p \leftarrow p + (1-fp)\cdot ( news - p ) $. En effet, plus sa force de persuasion sera faible, plus il sera enclin à croire à la news et sa conviction tendra alors plus fortement vers la valeur de la crédibilité de la news.
    - l'agent est également affecté par la **fréquence d'exposition antérieure** à la news, c'est-à-dire le nombre de fois où il a été confronté à la news par le biais de quelqu'un qui y croit. L'agent peut ainsi mesurer combien de personnes l'ont partagée en y croyant et pas en la démentant.

In [83]:
#Influence crébibilité news sur le p d'une personne

def influ_credib_news(news,p,fp):
    #news: crédibilité de la news
    i = p+(1-fp)*(news-p)
    return i

print(influ_credib_news(news,0.5,0.1),'\n')

def influ_partage(pop,res_virtu,partageur): #pop to get op,deja,fp,p ; res_virtu to get liste partage

    Liste_partage=res_virtu[partageur]
    op_p,deja_p,fp_p,p_p=pop[partageur]
    
    if deja_p==0:
        return pop
    
    P=[]   #liste des convictions des membres de la liste partage
    Fp=[]  #liste des forces de persuasion des membres de la liste partage
    i=0
    
    for k in Liste_partage:
        op_k,deja_k,fp_k,p_k=pop[k]
        P.append(p_k)
        Fp.append(fp_k)
        
    for a in Liste_partage:
        op_a,deja_a,fp_a,p_a=pop[a]
        if 'X' not in a:
            
        if fp_p>fp_a:
            p_a=p_a+fp_p*(p_p-p_a)
        p_a=influ_credib_news(news,p_a,fp_a)
        if op_p==1:
            deja_a=deja_a+1
        pop[a]=op_a,deja_a,fp_a,p_a
    
    return pop

Partageur=np.random.choice(Noms_agents_avec)
print('Pop avant partage \n',Pop_ac_om,'\n')
print('Celui qui partage l\'info :', Partageur,'\n')
print('Amis de celui qui partage :', Reseau_virtuel[Partageur],'\n')
print('Pop après partage \n', influ_partage(Pop_ac_om, Reseau_virtuel, Partageur))

0.8121759884833691 

Pop avant partage 
 {'2': (1, 36, 0.31173995939305765, 0.79594815840461686), '3': (1, 28, 0.39821370480178908, 0.77506504141734867), '4': (1, 38, 0.89946181454047647, 0.76807808257680565), '5': (1, 34, 0.5673332091959512, 0.80394823566513374), '6': (1, 38, 0.95670476510005531, 0.77552392514486568), '7': (1, 26, 0.41974855246685594, 0.77544960842130672), '9': (1, 14, 0.51252196700867259, 0.80920469198193468), '12': (1, 34, 0.92086253788237671, 0.76851716882934573), '15': (1, 29, 0.056496006421817069, 0.80662576333507441), '16': (1, 35, 0.55026588433457202, 0.80751140662483445), '18': (1, 27, 0.38599295526924293, 0.7721764194248667), '19': (1, 44, 0.7076605156622775, 0.7446398099086593), '22': (1, 37, 0.48678233038112906, 0.76782500845574786), '23': (1, 39, 0.51044859077003224, 0.77260127126363431), '25': (1, 23, 0.86690371366209029, 0.78132508134678558), 'OM1': (1, 32, 0.8433326284837656, 0.7822720582133198), 'XOM8': (1, 17, 0.9591723930751845, 0.7253281488538623), 

In [91]:
#Lancement fake news
#choix aléatoire d'une média malveillant de la liste_xom comme le départ de la propagation du fake news

def lancement_fake_news(pop,list_xom):
    lanceur='XOM'+str(np.random.choice(list_xom))
    pop[lanceur]=(1,1,np.random.uniform(low=0.7,high=1.0),0.75)
    for xom in list_xom:
        if 'XOM'+str(xom)!=lanceur:
            pop['XOM'+str(xom)]=(0,0,np.random.uniform(low=0.7,high=1.0),0.75)
    return pop,lanceur

Pop_ap_lan,Lanceur=lancement_fake_news(Pop_ac_om, List_xom)
print('Lanceur :',Lanceur)
print('Population après le lancement du fake news : \n',Pop_ap_lan)

Lanceur : XOM6
Population après le lancement du fake news : 
 {'1': (0, 0, 0.19142537518546299, 0.5), '2': (0, 0, 0.42617645726262621, 0.5), '4': (0, 0, 0.53636431730538447, 0.5), '10': (0, 0, 0.48662593051871961, 0.5), '11': (0, 0, 0.37981409874306105, 0.5), '12': (0, 0, 0.031889091784432222, 0.5), '15': (0, 0, 0.1390298493992248, 0.5), '16': (0, 0, 0.40817740745302278, 0.5), '18': (0, 0, 0.84434427237852649, 0.5), '19': (0, 0, 0.28016957297152989, 0.5), '20': (0, 0, 0.80745085660549609, 0.5), '21': (0, 0, 0.051135103432193629, 0.5), '23': (0, 0, 0.82429456181790595, 0.5), '24': (0, 0, 0.036613062216167891, 0.5), '25': (0, 0, 0.89415656729113968, 0.5), 'XOM3': (0, 0, 0.7457649073794915, 0.75), 'OM5': (0, 0, 0.9340386469605834, 0.5), 'XOM6': (1, 1, 0.9267878433280312, 0.75), 'OM7': (0, 0, 0.7347545330621289, 0.5), 'OM8': (0, 0, 0.9747503664193358, 0.5), 'OM9': (0, 0, 0.894920531412423, 0.5), 'OM13': (0, 0, 0.8971635020492176, 0.5), 'XOM14': (0, 0, 0.8612421750756183, 0.75), 'XOM17': (0

# II-) Simulation

In [63]:
Reseau_discu={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Pop_ac_om=init_ac_om_et_xom(Reseau_discu,List_om,List_xom)
print('Matrice population :\n',Mat_pop_ac_om)

def simulation_op_distance(pop,res_virtu,names,n_1dim,steps,list_xom,mat_pop):
    """retourne la matrice des OPINIONS de la population après un certain nb d'étapes"""
    
    pop_ap_lan,lanceur=lancement_fake_news(pop,list_xom)
    results=[matrice_opinions(pop,names,n_1dim)]
    print('Lanceur :',lanceur)
    print('Liste d\'abonnés du lanceur :',res_virtu[lanceur])
    
    for simu in range(steps):
        if simu==0:
            i=lanceur
        else:
            i=np.random.choice(names)
        Res_p_changé=influ_partage(pop,res_virtu,i)
        for c in Res_p_changé:
            op,deja,fp,p=Res_p_changé[c]
            if p>0.5 and op==0:
                op=1
                Res_p_changé[c]=(op,deja,fp,p)
            elif p<0.5 and op==1:
                op=0
                Res_p_changé[c]=(op,deja,fp,p)
        Op=[]
        for e in mat_pop.ravel():
            op,deja,fp,p=Res_p_changé[e]
            Op.append(op)
        Op_final=np.reshape(Op, (n_1dim,n_1dim))
        results.append(Op_final.copy())
    return results

Simulation_op=simulation_op_distance(Pop_ac_om,Reseau_virtuel,Noms_agents_avec,N_1dim,steps,List_xom,Mat_pop_ac_om)
Simulation_op

Matrice population :
 [['9' 'OM13' '2' '3' '11']
 ['19' 'OM12' '16' '5' '4']
 ['XOM14' 'OM6' '8' 'OM17' '22']
 ['7' '23' 'OM24' '10' 'OM1']
 ['15' 'XOM20' 'XOM21' 'XOM25' '18']]
Lanceur : XOM14
Liste d'abonnés du lanceur : ['8', '3', '23', '11', '18', '4', 'OM12', '9', 'OM13', '2', '19', 'OM17', '22', 'OM24', 'OM1', '15', 'XOM20', 'XOM25']


[array([[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]]), array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0]]), array([[0, 0, 1, 0, 0],
        [1, 1, 0, 0, 1],
        [1, 0, 1, 0, 0],
        [1, 1, 0, 0, 1],
        [0, 1, 1, 1, 1]]), array([[0, 0, 1, 0, 0],
        [1, 1, 0, 0, 1],
        [1, 0, 1, 0, 0],
        [1, 1, 0, 0, 1],
        [0, 1, 1, 1, 1]]), array([[0, 0, 1, 0, 0],
        [1, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 1, 1]]), array([[1, 0, 1, 1, 0],
        [1, 1, 1, 0, 1],
        [1, 0, 1, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 1, 1, 1]]), array([[1, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 1, 1, 1]]), array([[0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 

In [96]:
Reseau_discu={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Pop_ac_om=init_ac_om_et_xom(Reseau_discu,List_om,List_xom)
print('Matrice population :\n',Mat_pop_ac_om)

def simulation_p_distance(pop,res_virtu,names,n_1dim,steps,list_xom,mat_pop):
    """retourne la matrice des CONVICTIONS de la population après un certain nb d'étapes"""
    
    pop_ap_lan,lanceur=lancement_fake_news(pop,list_xom)
    results=[matrice_convic(pop_ap_lan,names,n_1dim)]
    print('Lanceur :',lanceur)
    print('Liste d\'abonnés du lanceur :',res_virtu[lanceur])
    
    for simu in range(steps):
        if simu==0:
            i=lanceur
        else:
            i=np.random.choice(names)
        Res_p_changé=influ_partage(pop,res_virtu,i)
        for c in Res_p_changé:
            op,deja,fp,p=Res_p_changé[c]
            if p>0.5 and op==0:
                op=1
                Res_p_changé[c]=(op,deja,fp,p)
            elif p<0.5 and op==1:
                op=0
                Res_p_changé[c]=(op,deja,fp,p)
        P=[]
        for e in mat_pop.ravel():
            op,deja,fp,p=Res_p_changé[e]
            P.append(p)
        P_final=np.reshape(P, (n_1dim,n_1dim))
        results.append(P_final.copy())
        
    return results

Simulation_p=simulation_p_distance(Pop_ac_om,Reseau_virtuel,Noms_agents_avec,N_1dim,steps,List_xom,Mat_pop_ac_om)
Simulation_p

Matrice population :
 [['14' '18' 'XOM5' 'OM24' '16']
 ['OM17' 'XOM23' 'XOM25' '4' 'XOM3']
 ['OM1' '7' 'OM8' '6' '2']
 ['15' '10' 'OM11' '21' '20']
 ['12' 'OM13' '19' '22' '9']]
Lanceur : XOM3
Liste d'abonnés du lanceur : ['18', 'OM11', 'OM17', '2', 'XOM23', '6', '4', '19', '20', 'XOM25', '21', 'XOM5', '16', 'OM1', '7', 'OM8', '15', '10', '9']


[array([[ 0.5 ,  0.5 ,  0.75,  0.5 ,  0.5 ],
        [ 0.5 ,  0.75,  0.75,  0.5 ,  0.75],
        [ 0.5 ,  0.5 ,  0.5 ,  0.5 ,  0.5 ],
        [ 0.5 ,  0.5 ,  0.5 ,  0.5 ,  0.5 ],
        [ 0.5 ,  0.5 ,  0.5 ,  0.5 ,  0.5 ]]),
 array([[ 0.5       ,  0.37147304,  0.73923101,  0.5       ,  0.29660051],
        [ 0.48222101,  0.71912303,  0.71674883,  0.56786871,  0.75      ],
        [ 0.45161983,  0.49208107,  0.49199088,  0.44131587,  0.46404787],
        [ 0.3106065 ,  0.51617647,  0.49754563,  0.32450826,  0.47798653],
        [ 0.5       ,  0.5       ,  0.49713486,  0.5       ,  0.45084434]]),
 array([[ 0.5       ,  0.37147304,  0.72871433,  0.5       ,  0.29660051],
        [ 0.48222101,  0.71912303,  0.4853703 ,  0.56786871,  0.44263111],
        [ 0.45161983,  0.39474947,  0.49199088,  0.43415469,  0.46404787],
        [ 0.3106065 ,  0.51617647,  0.49512   ,  0.32450826,  0.47798653],
        [ 0.5       ,  0.5       ,  0.49713486,  0.5       ,  0.37276112]]),
 array([[ 0.5      

In [97]:
Reseau_discu={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Pop_ac_om=init_ac_om_et_xom(Reseau_discu,List_om,List_xom)
print('Matrice population :\n',Mat_pop_ac_om)

def simu_op_finale(pop,res_virtu,names,n_1dim,steps,list_xom,mat_pop):

    pop_ap_lan,lanceur=lancement_fake_news(pop,list_xom)
    results=[matrice_opinions(pop,names,n_1dim)]
    print('Lanceur :',lanceur)
    print('Liste d\'abonnés du lanceur :',res_virtu[lanceur])
    
    for simu in range(steps):
        if simu==0:
            i=lanceur
            Res_p_changé=influ_partage(pop,res_virtu,i)          
        else:
            i=np.random.choice(names)
            if simu%2==1:
                list_discu=selection_voisinage(mat_pop,n_1dim,i)
                for a in list_discu:
                    if 'O' in a:
                        list_discu=list_discu[list_discu!=a]
                if len(list_discu)>=2:
                    Res_p_changé=influ_voisinage(list_discu,pop,seuil)
            else:
                Res_p_changé=influ_partage(pop,res_virtu,i)          

        for c in Res_p_changé:
            op,deja,fp,p=Res_p_changé[c]
            if p>0.5 and op==0:
                op=1
                Res_p_changé[c]=(op,deja,fp,p)
            elif p<0.5 and op==1:
                op=0
                Res_p_changé[c]=(op,deja,fp,p)
        Op=[]
        for e in mat_pop.ravel():
            op,deja,fp,p=Res_p_changé[e]
            Op.append(op)
        Op_final=np.reshape(Op, (n_1dim,n_1dim))
        results.append(Op_final.copy())
    return results

Simu_op_finale=simu_op_finale(Pop_ac_om,Reseau_virtuel,Noms_agents_avec,N_1dim,steps,List_xom,Mat_pop_ac_om)
Simu_op_finale

Matrice population :
 [['14' '18' 'XOM5' 'OM24' '16']
 ['OM17' 'XOM23' 'XOM25' '4' 'XOM3']
 ['OM1' '7' 'OM8' '6' '2']
 ['15' '10' 'OM11' '21' '20']
 ['12' 'OM13' '19' '22' '9']]
Lanceur : XOM5
Liste d'abonnés du lanceur : ['2', '4', '19', '15', 'XOM3', '9', 'OM24', 'XOM23', 'OM8', '21', '20', '12']


[array([[ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]]), array([[0, 0, 1, 1, 0],
        [0, 1, 1, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), array([[0, 0, 1, 1, 0],
        [0, 1, 1, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), array([[0, 0, 1, 1, 0],
        [0, 1, 1, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), array([[0, 0, 1, 1, 0],
        [0, 1, 1, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), array([[0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), array([[0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), array([[0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 

In [98]:
Reseau_discu={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Pop_ac_om=init_ac_om_et_xom(Reseau_discu,List_om,List_xom)
print('Matrice population :\n',Mat_pop_ac_om)

def simu_p_finale(pop,res_virtu,names,n_1dim,steps,list_xom,mat_pop):

    pop_ap_lan,lanceur=lancement_fake_news(pop,list_xom)
    results=[matrice_convic(pop,names,n_1dim)]
    print('Lanceur :',lanceur)
    print('Liste d\'abonnés du lanceur :',res_virtu[lanceur])
    
    for simu in range(steps):
        if simu==0:
            i=lanceur
            Res_p_changé=influ_partage(pop,res_virtu,i)          
        else:
            i=np.random.choice(names)
            k=np.random.choice([0,1])
            if k==0:
                list_discu=selection_voisinage(mat_pop,n_1dim,i)
                for a in list_discu:
                    if 'O' in a:
                        list_discu=list_discu[list_discu!=a]
                if len(list_discu)>=2:
                    Res_p_changé=influ_voisinage(list_discu,pop,seuil)
            else:
                Res_p_changé=influ_partage(pop,res_virtu,i)          

        for c in Res_p_changé:
            op,deja,fp,p=Res_p_changé[c]
            if p>0.5 and op==0:
                op=1
                Res_p_changé[c]=(op,deja,fp,p)
            elif p<0.5 and op==1:
                op=0
                Res_p_changé[c]=(op,deja,fp,p)
        P=[]
        for e in mat_pop.ravel():
            op,deja,fp,p=Res_p_changé[e]
            P.append(p)
        P_final=np.reshape(P, (n_1dim,n_1dim))
        results.append(P_final.copy())
    return results

Simu_p_finale=simu_p_finale(Pop_ac_om,Reseau_virtuel,Noms_agents_avec,N_1dim,steps,List_xom,Mat_pop_ac_om)
Simu_p_finale

Matrice population :
 [['14' '18' 'XOM5' 'OM24' '16']
 ['OM17' 'XOM23' 'XOM25' '4' 'XOM3']
 ['OM1' '7' 'OM8' '6' '2']
 ['15' '10' 'OM11' '21' '20']
 ['12' 'OM13' '19' '22' '9']]
Lanceur : XOM5
Liste d'abonnés du lanceur : ['2', '4', '19', '15', 'XOM3', '9', 'OM24', 'XOM23', 'OM8', '21', '20', '12']


[array([[ 0.5 ,  0.5 ,  0.75,  0.5 ,  0.5 ],
        [ 0.5 ,  0.75,  0.75,  0.5 ,  0.75],
        [ 0.5 ,  0.5 ,  0.5 ,  0.5 ,  0.5 ],
        [ 0.5 ,  0.5 ,  0.5 ,  0.5 ,  0.5 ],
        [ 0.5 ,  0.5 ,  0.5 ,  0.5 ,  0.5 ]]),
 array([[ 0.5       ,  0.5       ,  0.75      ,  0.47327324,  0.5       ],
        [ 0.5       ,  0.63742726,  0.75      ,  0.38346284,  0.6947266 ],
        [ 0.5       ,  0.5       ,  0.61213366,  0.5       ,  0.5122794 ],
        [ 0.29415279,  0.5       ,  0.5       ,  0.5781705 ,  0.46656394],
        [ 0.59167394,  0.5       ,  0.42991559,  0.5       ,  0.49098238]]),
 array([[ 0.5       ,  0.5       ,  0.75      ,  0.47327324,  0.5       ],
        [ 0.5       ,  0.63742726,  0.75      ,  0.38346284,  0.6947266 ],
        [ 0.5       ,  0.5       ,  0.61213366,  0.5       ,  0.5122794 ],
        [ 0.29415279,  0.5       ,  0.5       ,  0.5781705 ,  0.46656394],
        [ 0.59167394,  0.5       ,  0.42991559,  0.5       ,  0.49098238]]),
 array([[ 0.5      

In [79]:
#Affichage des opinions de départ
Opinions_start=Simulation_op[0]
fig=plt.figure()
im=plt.imshow(Opinions_start,cmap='binary',vmin=0,vmax=1)

#Affichage des opinions d'arrivée
Opinions_end=Simulation_op[-1]
fig=plt.figure()
im=plt.imshow(Opinions_end,cmap='binary',vmin=0,vmax=1)

#blanc = 0 ; noir = 1

In [14]:
#Affichage des convictions de départ
Convictions_start=Simulation_p[0]
fig=plt.figure()
im=plt.imshow(Convictions_start,cmap='bwr',vmin=0,vmax=1)
plt.colorbar()

#Affichage des convictions de fin de simulation

Convictions_end=Simulation_p[-1]
fig=plt.figure()
im=plt.imshow(Convictions_end,cmap='bwr',vmin=0,vmax=1)
plt.colorbar()